In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 28.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from konlpy.tag import Okt  # KoNLPy의 형태소 분석기

In [4]:
# Google Drive와 연동
from google.colab import drive
drive.mount('/content/drive')

# 데이터 로드 및 병합
files = ['/content/drive/MyDrive/지인_categorized.csv',
         '/content/drive/MyDrive/택배_categorized.csv',
         '/content/drive/MyDrive/이벤트_categorized.csv',
         '/content/drive/MyDrive/채용_categorized.csv',
         '/content/drive/MyDrive/금융_categorized.csv',
         '/content/drive/MyDrive/기타_categorized.csv',
         '/content/drive/MyDrive/기관_categorized.csv']

dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)


Mounted at /content/drive


In [5]:
# 전처리: 불용어 리스트와 함께 처리
okt = Okt()
stopwords = ['은', '는', '이', '가', '을', '를', '에', '의', '도', '으로', '하다']  # 예시 불용어 리스트

def preprocess_text(text):
    tokens = okt.morphs(text, stem=True)  # 형태소 분석
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return ' '.join(tokens)

df['processed_text'] = df['v2'].apply(preprocess_text)  # 'v2' 컬럼을 텍스트로 사용

# TF-IDF 변환
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['processed_text'])

# 라벨 데이터 준비
y = df['category']

# 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 학습 (확률 계산을 위해 probability=True 설정)
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

          금융       0.98      1.00      0.99      1391
          기관       0.99      0.96      0.98       448
          기타       0.97      0.99      0.98       530
         이벤트       1.00      0.67      0.80        30
          지인       1.00      1.00      1.00     15622
          채용       0.00      0.00      0.00        18
          택배       0.92      0.91      0.92       161

    accuracy                           1.00     18200
   macro avg       0.84      0.79      0.81     18200
weighted avg       0.99      1.00      1.00     18200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
# 실제 텍스트 입력 테스트
def predict_text(text, threshold=0.5):
    # 입력 텍스트 전처리
    processed = preprocess_text(text)
    vectorized = tfidf.transform([processed])

    # 예측 확률 계산
    prob = model.predict_proba(vectorized)[0][1]  # 스미싱일 확률 (1에 대한 확률)

    # 결과 출력
    if prob >= threshold:
        print(f"스미싱 의심: {prob * 100:.2f}% 확률로 스미싱")
    else:
        print(f"스미싱 아님: {prob * 100:.2f}% 확률로 스미싱 아님")

# 사용자로부터 텍스트 입력받기
user_input = input("확인할 메시지를 입력하세요: ")
predict_text(user_input)

확인할 메시지를 입력하세요: 엄마 나 교통사고가 났는데 핸드폰은 잃어버렸고 내 친구가 입원비 대신 내줬거든   100124-56-094740 여기로 10만원만 보내줘 친구가 대신 내줬어
스미싱 아님: 0.00% 확률로 스미싱 아님
